In [1]:
import pandas as pd
import os
import sys

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "processed")

# [입력 1] 방금 만든 주가 6종 통합본
MASTER_FILE = os.path.join(DATA_DIR, "master.csv")
# [입력 2] 1단계에서 Ticker 통일한 재무 데이터
FINANCIALS_FILE = os.path.join(DATA_DIR, "financials_mapped.csv")
# [입력 3] 이전에 만든 거시 경제 데이터
MACRO_FILE = os.path.join(DATA_DIR, "macro_data.csv")

print(f"Base file (Price): {MASTER_FILE}")
print(f"Merging file (Financials): {FINANCIALS_FILE}")
print(f"Merging file (Macro): {MACRO_FILE}")

# --- 1. [Base] 주가 6종 통합 데이터 로드 ---
try:
    master_df = pd.read_csv(MASTER_FILE, parse_dates=['date'])
    master_df = master_df.sort_values(by='date') # As-of 병합을 위해 날짜 정렬
    print(f"\nLoaded master_df (Price data): {len(master_df)} rows")
    
except FileNotFoundError:
    print(f"\n[!!!] Error: {MASTER_FILE}을 찾을 수 없습니다.")
    print("이전 단계(Phase 1-B)를 먼저 실행해주세요.")
    # (에러 발생 시 중단)
    sys.exit()

# --- 2. [Phase 1-A] 재무 데이터(financials_mapped) 'As-of' 병합 ---
try:
    financials_df = pd.read_csv(FINANCIALS_FILE, parse_dates=['as_of_date'])
    financials_df = financials_df.sort_values('as_of_date') # As-of 병합을 위해 날짜 정렬
    
    # [핵심] As-of 병합 (기업별로, 날짜 기준으로)
    master_df = pd.merge_asof(
        master_df,
        financials_df[['as_of_date', 'ticker', 'roe', 'real_debt_ratio']], # 필요한 컬럼만 선택
        left_on='date',
        right_on='as_of_date',
        by='ticker',           # [중요] 기업별(ticker)로 그룹지어 병합
        direction='backward'   # 리크 방지 (과거/현재 값만 보기)
    )
    
    print("  > Merged financials_mapped.csv (as-of)")
    
except FileNotFoundError:
    print(f"\n[!!!] Error: {FINANCIALS_FILE}을 찾을 수 없습니다.")
    print("1단계(Ticker 통일)를 먼저 실행하고 'financials_mapped.csv'로 저장했는지 확인해주세요.")
    sys.exit()

# --- 3. [Phase 1-C] 거시 경제 데이터(macro_data) 병합 ---
try:
    macro_df = pd.read_csv(MACRO_FILE, parse_dates=['date'])
    
    # [핵심] 날짜 기준으로 단순 병합 (how='left')
    master_df = pd.merge(master_df, macro_df, on='date', how='left')
    
    print("  > Merged macro_data.csv (on 'date')")
    
    # [핵심 ffill] 거시 데이터는 모든 기업에 공통 적용되므로, 
    # ffill() 전에 'ticker' 기준으로 정렬해야 그룹별로 ffill이 오염되지 않습니다.
    master_df = master_df.sort_values(by=['ticker', 'date'])
    
    macro_cols = ['wti', 'bdi_proxy', 'newbuild_proxy_2015_100']
    
    # [중요] groupby('ticker')를 사용하여 기업별로 ffill 수행
    master_df[macro_cols] = master_df.groupby('ticker')[macro_cols].ffill()
    
    print("  > Forward-filled macro data columns (grouped by ticker)")

except FileNotFoundError:
    print(f"\n[!!!] Error: {MACRO_FILE}을 찾을 수 없습니다.")
    print("이전 단계(macro_data 생성)를 먼저 실행해주세요.")
    sys.exit()

# --- 4. 결과 확인 ---
print("\n[SUCCESS] Phase 1-C (Financials + Macro) 병합 완료!")
print(f"Total rows in master_df: {len(master_df)}")

print("\n--- master_df (Head) ---")
print(master_df.head())

print("\n--- master_df (Tail) ---")
print(master_df.tail())

print("\n--- master_df (Columns) ---")
print(master_df.info())

Base file (Price): /workspace/ship-ai/data/processed/master.csv
Merging file (Financials): /workspace/ship-ai/data/processed/financials_mapped.csv
Merging file (Macro): /workspace/ship-ai/data/processed/macro_data.csv

Loaded master_df (Price data): 6872 rows
  > Merged financials_mapped.csv (as-of)
  > Merged macro_data.csv (on 'date')
  > Forward-filled macro data columns (grouped by ticker)

[SUCCESS] Phase 1-C (Financials + Macro) 병합 완료!
Total rows in master_df: 6872

--- master_df (Head) ---
         date    open    high     low   close  trading_volume  ticker  \
0  2019-01-02  129500  131500  125000  125500          143838    9540   
6  2019-01-03  126500  133000  126000  131500          334941    9540   
10 2019-01-04  132000  139500  130500  138500          369927    9540   
13 2019-01-07  140500  143000  137500  139000          247943    9540   
19 2019-01-08  140000  140000  133500  134000          247213    9540   

   as_of_date  roe  real_debt_ratio        wti  bdi_proxy  

In [2]:
import os

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "processed")

# 저장할 파일 경로
SAVE_FILE_PATH = os.path.join(DATA_DIR, "master_c_result.csv")

# --- 1. 메모리의 master_df를 파일로 저장 ---
try:
    if 'master_df' in locals():
        master_df.to_csv(SAVE_FILE_PATH, index=False)
        print(f"[SUCCESS] Phase 1-C 결과가 파일로 저장되었습니다!")
        print(f"  -> {SAVE_FILE_PATH}")
    else:
        print("[Error] 'master_df'가 메모리에 없습니다.")
        print("이전 단계(Phase 1-C) 셀을 다시 실행해주세요.")
        
except Exception as e:
    print(f"[Error] 파일 저장 중 오류 발생: {e}")

[SUCCESS] Phase 1-C 결과가 파일로 저장되었습니다!
  -> /workspace/ship-ai/data/processed/master_c_result.csv
